# Spotify Insights
note: The value of streams is NULL when the chart column is "viral50".

In [1]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
import time


In [2]:

# data = pd.read_csv('charts.csv')
# kernel chashed

start = time.time()
#read data in chunks of 1 million rows at a time
chunk = pd.read_csv('charts.csv',chunksize=1000000)
end = time.time()
data = pd.concat(chunk)
data.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


### Definição da tabela de dados


##### Quantidade de linhas e colunas

In [3]:
data.shape

(26173514, 9)

##### Tratamento de dados
- Alteração do tipo de dados das colunas

In [4]:

spark = (SparkSession.builder.config("spark.driver.memory","4g").config("spark.driver.maxResultSize", "4g").getOrCreate())
# import dataframe by spark
df = spark.read.csv(path='charts.csv', inferSchema=True, header=True)
df = df.withColumn("rank", f.col("rank").cast(t.IntegerType())).withColumn("date", f.col("date").cast(t.DateType())).withColumn("streams", f.col("streams").cast(t.IntegerType()))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/06 21:08:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/06 21:08:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


##### Verificação de duplicatas

In [5]:
data.duplicated().any()
duplicated = data.duplicated()

: 

: 

In [ ]:
data[duplicated]

##### Dados nulos

In [ ]:
data.isnull().sum()

In [ ]:
data.fillna(0)
data.shape

In [ ]:

import matplotlib.pyplot as plt
from wordcloud import WordCloud

def visualize_word_counts(counts):
    wc = WordCloud(max_font_size=130, min_font_size=25, colormap='tab20', background_color='white', 
                   prefer_horizontal=.95, width=2100, height=700, random_state=0)
    cloud = wc.generate_from_frequencies(counts)
    plt.figure(figsize=(18,15))
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()
    
counts = data['artist'].value_counts()
visualize_word_counts(counts)

### Prever proximo artista a ficar top 200 no brasil